In [1]:
%matplotlib inline

import torch
import numpy as np
import pandas as pd
import seaborn as sns

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

from torch.autograd import Variable
from torchvision import datasets, transforms

sns.set()

In [470]:
transform = transforms.Compose([
                       transforms.ToTensor()])

MNIST_train = datasets.MNIST(r'D:\Data_sets/MNIST', 
                            train=True, download=True,
                            transform=transform)

MNIST_test = datasets.MNIST(r'D:\Data_sets/MNIST', 
                            train=False, download=True,
                            transform=transform)

train_loader = torch.utils.data.DataLoader(
                            MNIST_train, batch_size=64, 
                            shuffle=True, pin_memory=True)

test_loader = torch.utils.data.DataLoader(MNIST_test,
                            batch_size=1000, shuffle=True, 
                            pin_memory=True)

In [471]:
img = next(iter(test_loader))[0][0]

In [2]:
class NNet(nn.Module):
    def __init__(self):
        super(NNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        #self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [474]:
model = NNet()

In [475]:
model

NNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [232]:
def mutate(x, chance, mean, std):
    if np.random.rand() > (1-chance): 
        x = np.random.normal(loc=mean, scale=std)
        return x
    return x

In [393]:
def mutate_weights(params, chance):
    mean = params.detach().mean()
    std = params.detach().std()
    return params.detach().apply_(lambda x: mutate(x, chance, m, s))

In [432]:
def update_weights(model, chance):
    params = list(model.parameters())
    for param in params:
        if len(param.shape) == 1:
            param.data = mutate_weights(param, chance)
        elif len(param.shape) == 2:
            for inner_param in param:
                    inner_param.data = mutate_weights(inner_param, chance)
        else:
            for inner_param in param:
                for last_channel in inner_param:
                    last_channel.data = mutate_weights(last_channel, chance)

In [715]:
def crossover(model1, model2):
    model1_params = list(model1.parameters())
    model2_params = list(model2.parameters())
    
    for i, param in enumerate(model1_params):
        if len(param.shape) == 1:
            split = int(np.random.uniform(low=0, high=param.shape[0]))
            first_half = param.view(-1)[:split].data 
            second_half = model2_params[i].view(-1)[split:].data
            comb = torch.cat((first_half, second_half)).view_as(param)
            first_half = param.view(-1)[split:].data
            second_half = model2_params[i].view(-1)[:split].data
            comb2 = torch.cat((first_half, second_half)).view_as(param)
            param.data = comb
            model2_params[i].data = comb2
        
        elif len(param.shape) == 2:
             for j, inner_param in enumerate(param):
                split = int(np.random.uniform(low=0, high=inner_param.shape[0]))
                first_half = inner_param.view(-1)[:split].data 
                second_half = model2_params[i][j].view(-1)[split:].data
                comb = torch.cat((first_half, second_half)).view_as(inner_param)
                first_half = inner_param.view(-1)[split:].data
                second_half = model2_params[i][j].view(-1)[:split].data
                comb2 = torch.cat((first_half, second_half)).view_as(inner_param)
                inner_param.data = comb
                model2_params[i][j].data = comb2

        else:
            for j, inner_param in enumerate(param):
                for k, last_channel in enumerate(inner_param):
                    split = int(np.random.uniform(low=0, 
                                high=last_channel.shape[0] * last_channel.shape[1]))
                    first_half = last_channel.view(-1)[:split].data 
                    second_half = model2_params[i][j][k].view(-1)[split:].data
                    comb = torch.cat((first_half, second_half)).view_as(last_channel)
                    first_half = last_channel.view(-1)[split:].data
                    second_half = model2_params[i][j][k].view(-1)[:split].data
                    comb2 = torch.cat((first_half, second_half)).view_as(last_channel)
                    last_channel.data = comb
                    model2_params[i][j][k].data = comb2